In [ ]:
import pandas as pd
import numpy as np
import os
import glob

curr_dir = os.getcwd()

# How Big is Canada, Really?

Canada is the second largest country in the world by area, but most of it is virtually uninhabited. In 2016, 66% of the Canadian population lived within [100km](https://www150.statcan.gc.ca/n1/daily-quotidien/170208/dq170208a-eng.htm) of the US border.
I've been curious for a long while now, how big is Canada really, if we only include area that has a certain population density or higher?

A naive approach would to go to every country's own statistics/census website, wrestle with their chosen data accessing and formatting patterns.
A slightly smarter approach is to let someone else do it for us, and use their results.
Luckily, the [Socioeconomic Data and Applications Center (sedac)](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4) has done this since 1995.

I originally downloaded their Population Density dataset v4.11, which contains ASCII and TIFF files of data up to a resolution of 30 arc-second.
Unfortunately, I couldn't think of a convenient way of figuring out which pixels belong to each country.
A much easier approach is to use the Administrative Unit Center Points with Population Estimates dataset, which gives, per administrative unit in a given country, its population, area, density, and more, which makes our job easy as pie.
There's definitely some resolution lost by solving this way, since a gigantic admin unit could hypothetically have everyone concentrated in a single square kilometer, and we'd never know.
Something to look out for when we start wrangling the data.

## Step 1: Download Data

I went over the the link above (or [here](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4), same link), created an account, selected Global/Regional as the Geography, Comma Separated Value as the file format, and then ticked the Global box, which contains a single CSV for the world minus the US, then four separate CSV for the latter. While ticking the per-continent boxes was an option, I'm here to make unbased claims against Mapleland, not parallelize my work or be considerate of the hardware of others. According to [Jeff](https://stackoverflow.com/questions/25962114/how-do-i-read-a-large-csv-file-with-pandas), it takes about double the size of a CSV file to open it in Pandas; since the largest file is just over 3GB, I didn't think it would cause any issues.

I also downloaded the documentation, since it describes the column titles used below.

## Step 2: Load, Trim, Permute, and Save

Next step was pretty straightforward; load each CSV, remove all columns but the country, density, and area (I kept population too for maybe futzing around later), and save it all to a CSV for probably faster loading if I ever want to open these again.

In [ ]:
files = glob.glob(curr_dir + '/data/unprocessed/*.csv')
combined_csv = pd.DataFrame()

for file in files:
    df = pd.read_csv(file)
    df = df[['COUNTRYNM', 'LAND_A_KM', 'UN_2020_E', 'UN_2020_DS']]
    combined_csv = pd.concat([combined_csv, df], ignore_index=True)

combined_csv.to_csv(curr_dir + '/data/processed/data.csv', index=False) # creative naming, I know

That trimmed the CSV file down to a much more manageable 700MB.
Let's load it back up and make sure we're in the right universe.

In [ ]:
processed_dir = curr_dir + '/data/processed/data.csv'
world = pd.read_csv(processed_dir)

canada = world.loc[world['COUNTRYNM'] == 'Canada']
print("Some basic stats to make sure we're in the right ballbark:\n" + canada[['LAND_A_KM', 'UN_2020_E']].sum().astype(int).to_string())
print("Largest countries by area:")
display(world.nlargest(10, 'LAND_A_KM'))

Population of 37 million and land area of 9 million? Seems about right.

Unfortunately, those administrative units are absolutely massive.
The Canadian one isn't too much trouble, since the population is zero, and there's no way to distribute zero people that will influence the density of any square kilimeter of land within it.

The others are more problematic.
The fourth largest, for example, contains 1.6 million people, but with an average density of 3.3 people per square kilometer, if our threshold were set to, for example, the average population density of the world (about [50/sqkm](https://en.wikipedia.org/wiki/Population_density)), then the entire area would be ignored.
Not much I can think about off the top of my head to fix this, so we'll just have to keep it in mind when looking at the results.

![World map with large administrative units visible](img/population_density_shrunk.jpg)

Smaller version of the 17GB TIFF from the Population Density dataset v4.11, with some very large administrative units visible.

## Step 3: How Dense is Dense Enough?

What density is enough to reasonably consider an administrative unit populated?
There's a few approaches we can take.

We can trim all units with a density below the global average of 50 people per square kilometer above; this'll hurt Canada's area quite a bit, but that's what we're here for.

We can also trim all units with a density below the average density in Canadian farmland. [Statistics Canada](https://www150.statcan.gc.ca/n1/pub/95-640-x/2011001/p1/p1-01-eng.htm) lists 160 155 748 acres of farmland in Canada in 2011, or 648 127sqkm. With about [241 500](https://agriculture.canada.ca/en/sector/overview) jobs in primary agriculture, that comes out to 2.68 people per square kilometer, significantly lower than the global average, and a generous threshold to let Canada keep some area in the upcoming smackdown.

We can look at the least dense US state and use that as our threshold. Why? No clue, seems like fun. [Wikipedia](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population_density) to the rescue, which reports Alaska as our lucky winner with an average population density of 0.5 people per sqkm. I was originally going to exclude Alaska, since I figured it would be ridiculously low, but considering the second lowest, Wyoming, has a density of 2.3 people per sqkm, which is about the same as for the average Canadian farm, looks like we're using our friend up north and being even more gentle with Canada's new area.

## Step 4: Shrink Canada

In [ ]:
world_thresh_alaska = world.loc[world['UN_2020_DS'] >= 0.5].drop('UN_2020_DS', axis=1).groupby('COUNTRYNM').sum().astype(int)
world_thresh_farming = world.loc[world['UN_2020_DS'] >= 2.7].drop('UN_2020_DS', axis=1).groupby('COUNTRYNM').sum().astype(int)
world_thresh_average = world.loc[world['UN_2020_DS'] >= 50].drop('UN_2020_DS', axis=1).groupby('COUNTRYNM').sum().astype(int)
world_unthresh = world.drop('UN_2020_DS', axis=1).groupby('COUNTRYNM').sum().astype(int)

thresholded_areas = pd.concat([world_unthresh['LAND_A_KM'],
                               world_thresh_alaska['LAND_A_KM'],
                               world_thresh_farming['LAND_A_KM'],
                               world_thresh_average['LAND_A_KM']],
                              axis=1,
                              keys = ['True Area', 'Alaska Threshold', 'Farming Threshold', 'Average Threshold'])

thresholded_populations = pd.concat([world_unthresh['UN_2020_E'],
                                     world_thresh_alaska['UN_2020_E'],
                                     world_thresh_farming['UN_2020_E'],
                                     world_thresh_average['UN_2020_E']],
                                    axis=1,
                                    keys = ['True Population', 'Alaska Threshold', 'Farming Threshold', 'Average Threshold'])
display(thresholded_areas)

While I could print out the entire Dataframe, it's pretty long and would cause some serious scrolling cramps, so I've hidden it down at the bottom.

Coutries who do not at all meet the threshold for any of their administrative units will return a NaN value, as demonstrated below for Belize, which has densities above that of both the Alaska and Farming thresholds, but not the world average one of 50. We'll fill those up with zeros instead.

In [ ]:
display(thresholded_areas[thresholded_areas['Average Threshold'].isnull()])
display(world.loc[world['COUNTRYNM'] == 'Belize'])

In [ ]:
# needed np.Nan instead of 'NaN', as per https://stackoverflow.com/questions/48956789/converting-nan-in-dataframe-to-zero
thresholded_areas = thresholded_areas.replace(np.NaN, 0).astype(int)
thresholded_populations = thresholded_populations.replace(np.NaN, 0).astype(int)

## Step 5: Get Our Bearings

The entry I'm really here for:

In [ ]:
thresholded_areas.loc['Canada']

Alaska's population of 0.5 people per square kilometer is so low that, were it a country/dependency, it would be the [fourth least dense](https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density) in the world after Greenland, Svalbard and Jan Mayen, and the Falkland Islands, and four times less dense than the next least dense, Mongolia. Even with such a generous threshold, Canada's area plummets by a factor of 10 down to 700 000 square kilometers; we're left with the provinces of [Alberta and Nova Scotia](https://en.wikipedia.org/wiki/Provinces_and_territories_of_Canada), or equivalently, [Texas or half of Alaska](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area)!

If we're slightly less polite, we can use a density threshold which matches that of a typically Canadian farm, Canada's area cuts in half again down to 320 000 square kilometers, so less than Newfoundland and Labrador, or New Mexico.

Finally, with a density threshold equal to the average population density over the entire landmass of the world, Canada's area disappears into thin air at 35 000 square kilometers. We have become the Netherlands. Or the New York metropolitan area three times (but with about half the population!)

Pitting Canada against itself and other un-harried nations does us little good though if they also shed some area, so lets see if Canada loses its number 2 spot when thresholded.

## Step 6: Bully Canada

First, lets find out how many countries are larger than Canada when using our very generous Alaskan threshold.

In [ ]:
larger_than_canada_alaska = thresholded_areas.loc[thresholded_areas['Alaska Threshold'] >= thresholded_areas.loc['Canada']['Alaska Threshold']].sort_values('Alaska Threshold', ascending=False)
print("Number of countries larger than Canada: ", larger_than_canada_alaska.shape[0] - 1)
display(larger_than_canada_alaska)

Such a small threshold, and yet here we are, already off the podium and forgotten. Have you ever thought to yourself "Wow, Zambia is such a large country"? Nope, neither have I! If we look at countries by true area, we're now the 40th largest. Lets make it worse.

In [ ]:
larger_than_canada_farming = thresholded_areas.loc[thresholded_areas['Farming Threshold'] >= thresholded_areas.loc['Canada']['Farming Threshold']].sort_values('Farming Threshold', ascending=False)
print("Number of countries larger than Canada: ", larger_than_canada_farming.shape[0] - 1)
display(larger_than_canada_farming)

Woohoo, even smaller! According to Google Maps, you can drive across Germany (from Flensburg to Garmisch-Partenkirchen) in about 11 hours. When I moved from Winnipeg to Ottawa for university, it took over twice that. We're now a little smaller than Germany as well, the 63rd largest country in the (unthresholded world). Finally, the death blow:

In [ ]:
larger_than_canada_average = thresholded_areas.loc[thresholded_areas['Average Threshold'] >= thresholded_areas.loc['Canada']['Average Threshold']].sort_values('Average Threshold', ascending=False)
print("Number of countries larger than Canada: ", larger_than_canada_average.shape[0] - 1)
print("Number of countries in total: ", thresholded_areas.shape[0])
display(larger_than_canada_average)

Excellent, we've surpassed the default number of rows that show in JupyterLab. We're now smaller than [Switzerland](https://en.wikipedia.org/wiki/Switzerland) at its true size, and sandwiched between Guinea-Bissau and Moldova to take the 139th (non-thresholded) largest country award.

![Map from freeworldmaps.net](img/switzerland.jpg)

It's so tiny you absolutely need a giant red circle to help you find it. Want to point to Guinea-Bissau on a map? You can't, because your finger is too big and will crush it, as well as some of the surrounding countries.

![Guinea Bissau](img/guineabissau.jpg)

Observe the smolness of the arrow. That might as well be pointing to Canada.

![Pathetic meme from Simpsons](img/pathetic.jpg)

I put the full table at the bottom of this document for your perusing pleasure.

## Step 7: Conclusions

What have we learned? Probably nothing, since as noted way above, many administrative units are absolutely massive with not-insignificant populations that could hypothetically contribute much needed square kilometers. If we're willing to forego any sense of critical thinking, we can safely conclude that Canada is actually a tiny country, at best the 30th largest, at worst a third of the way down the chain.

## Appendices: Full Dataframe

Here's the full dataframe containing areas at the various thresholds for all countries

In [ ]:
print("Full dataframe:")
with pd.option_context('display.max_rows', None):
    display(thresholded_areas)

print("Countries larger than Canada using average threshold:")
with pd.option_context('display.max_rows', None):
    display(larger_than_canada_average)